Investigating using Convolutional Networks on Weak Lensing data
=============

Implemented using tflearn


In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import division

import numpy as np
import time
import tensorflow as tf
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.normalization import local_response_normalization
from tflearn.layers.estimator import regression

from six.moves import cPickle as pickle
from six.moves import range
import matplotlib.pyplot as plt
from astropy.io import fits
plt.rcParams['image.cmap'] = 'viridis'
plt.rcParams['image.interpolation'] = 'none'
%matplotlib inline
from IPython import display

In [2]:
# Reorganize the code a bit by putting the function definitions first.
def rebin(a, shape):
    sh = shape[0],a.shape[0]//shape[0],shape[1],a.shape[1]//shape[1]
    return a.reshape(sh).mean(-1).mean(1)

def getFITS(imagename):
    filename = whereami + '/' + path + imagename
    f = fits.open(filename)
    dataout = f[0].data
    
    return dataout

def read_WL(path,display=None):
    # this is a version to look at sigma8
    labels=['750', '850']
    imgs = np.zeros([2048/degrade, 2048/degrade, nct, len(labels)])
    for j, label in enumerate(labels):
        for i in range(nct):
            filename = whereami + '/' + path + 'smoothWL-conv_m-512b240_Om0.260_Ol0.740_w-1.000_ns0.960_si0.'+label+'_4096xy_000'+ np.str(i+1) +'r_0029p_0100z_og.gre.fit'
            if display: print("i: %d  j: %d  name: %s" % (i, j, 'smoothWL-conv_m-512b240_Om0.260_Ol0.740_w-1.000_ns0.960_si0.'+label+'_4096xy_000'+ np.str(i+1) +'r_0029p_0100z_og.gre.fit'))
            f = fits.open(filename)
            imgs[:,:,i,j]=rebin(f[0].data, [2048/degrade, 2048/degrade])
            
    return imgs, labels

def slice_data(data, labels, exp_cut, exp_nshift):
    labels=['750', '850']
    # how many panels across
    npanelx = 2**exp_cut
    # and how big are they?
    panelw = 2048/(degrade*npanelx)
    # how many shifted panels?
    nshift = 2**exp_nshift -1
    # and what are the shifts?
    shiftw =  panelw/2**exp_nshift
    # with 4 rotations, and 2 shifts, we have
    imgs = np.zeros([panelw, panelw, nct,(npanelx**2 +(npanelx-1)**2*nshift**2)*8, len(labels)])
    # let's figure out where the centers are, and save that data
    x_centers = np.zeros([nct,(npanelx**2 +(npanelx-1)**2*nshift**2)*8, len(labels)])
    y_centers = np.zeros([nct,(npanelx**2 +(npanelx-1)**2*nshift**2)*8, len(labels)])
    for j, label in enumerate(labels):
        for i in range(nct):
            q=0
            for k in range(npanelx):
                for l in range(npanelx):
                    for r in range(4):
                        imgs[:,:,i,q,j] = np.rot90(data[panelw*k:panelw*(k+1),panelw*l:panelw*(l+1),i, j], r)
                        x_centers[i,q,j] = (panelw*k+panelw*(k+1))/2.
                        y_centers[i,q,j] = (panelw*l+panelw*(l+1))/2.
                        q+=1
                        imgs[:,:,i,q,j] = np.fliplr(np.rot90(data[panelw*k:panelw*(k+1),panelw*l:panelw*(l+1),i, j], r))
                        x_centers[i,q,j] = (panelw*k+panelw*(k+1))/2.
                        y_centers[i,q,j] = (panelw*l+panelw*(l+1))/2.
                        q+=1
            for k in range(npanelx-1):
                for l in range(npanelx-1):
                    for m in range(nshift):
                        for n in range(nshift):
                            for r in range(4):
                                imgs[:,:,i,q,j] = np.rot90(data[panelw*k+m*shiftw:panelw*(k+1)+m*shiftw,panelw*l+n*shiftw:panelw*(l+1)+n*shiftw,i, j], r)
                                x_centers[i,q,j] = (panelw*k+m*shiftw+panelw*(k+1)+m*shiftw)/2.
                                y_centers[i,q,j] = (panelw*l+n*shiftw+panelw*(l+1)+n*shiftw)/2.
                                q+=1
                                imgs[:,:,i,q,j] = np.fliplr(np.rot90(data[panelw*k+m*shiftw:panelw*(k+1)+m*shiftw,panelw*l+n*shiftw:panelw*(l+1)+n*shiftw,i, j], r))
                                x_centers[i,q,j] = (panelw*k+m*shiftw+panelw*(k+1)+m*shiftw)/2.
                                y_centers[i,q,j] = (panelw*l+n*shiftw+panelw*(l+1)+n*shiftw)/2.
                                q+=1
    return imgs, x_centers, y_centers

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [3]:
# Set the paths to the raw data files
#whereami = '/home/jhargis'
whereami = '/Users/jhargis'
path     = 'Dropbox/astroNN/wl_maps/'
#whereami = '/Users/goldston'
#whereami = '/Users/jegpeek'
#path = 'Documents/Weak_Lensing/kmaps_smoothed/'


In [4]:
# Set (1) the factor by which we want to degrade the original WL maps
# and (2) the number of realizations of each universe.
#
#   The original images are 2048 x 2048, and we degrade them using
#   an 8 x 8 sq.pix box, which makes a smaller set of 64 images 
#   (= 256x256 sq.pix in size).

degrade=8
nct = 9

data, labels = read_WL(path,display=True)

print "Data shape :",data.shape
print "Labels     :",labels

/Users/jhargis/anaconda/envs/tensorflow/lib/python2.7/site-packages/ipykernel/__main__.py:16: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/jhargis/anaconda/envs/tensorflow/lib/python2.7/site-packages/ipykernel/__main__.py:4: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


i: 0  j: 0  name: smoothWL-conv_m-512b240_Om0.260_Ol0.740_w-1.000_ns0.960_si0.750_4096xy_0001r_0029p_0100z_og.gre.fit
i: 1  j: 0  name: smoothWL-conv_m-512b240_Om0.260_Ol0.740_w-1.000_ns0.960_si0.750_4096xy_0002r_0029p_0100z_og.gre.fit
i: 2  j: 0  name: smoothWL-conv_m-512b240_Om0.260_Ol0.740_w-1.000_ns0.960_si0.750_4096xy_0003r_0029p_0100z_og.gre.fit
i: 3  j: 0  name: smoothWL-conv_m-512b240_Om0.260_Ol0.740_w-1.000_ns0.960_si0.750_4096xy_0004r_0029p_0100z_og.gre.fit
i: 4  j: 0  name: smoothWL-conv_m-512b240_Om0.260_Ol0.740_w-1.000_ns0.960_si0.750_4096xy_0005r_0029p_0100z_og.gre.fit
i: 5  j: 0  name: smoothWL-conv_m-512b240_Om0.260_Ol0.740_w-1.000_ns0.960_si0.750_4096xy_0006r_0029p_0100z_og.gre.fit
i: 6  j: 0  name: smoothWL-conv_m-512b240_Om0.260_Ol0.740_w-1.000_ns0.960_si0.750_4096xy_0007r_0029p_0100z_og.gre.fit
i: 7  j: 0  name: smoothWL-conv_m-512b240_Om0.260_Ol0.740_w-1.000_ns0.960_si0.750_4096xy_0008r_0029p_0100z_og.gre.fit
i: 8  j: 0  name: smoothWL-conv_m-512b240_Om0.260_Ol0.74

In [5]:
# First we slice the data in the following manner:
#
#   1) Each 256 x 256 image is again split 8 x 8 into 64 images which are 32 x 32 sq. pix in size
#   2) Series of 4 rotations, 2 flips, and shifts of 2 pixels in size
imgs2, x_centers, y_centers = slice_data(data, labels, 3, 3)
img2sh = imgs2.shape

# Next, reshape the arrays and take the first 7 realizations as the training data set
train_dataset = np.transpose(imgs2[:, :, 0:7, :, :].reshape(img2sh[0], img2sh[1], 7.0*img2sh[3]*2.0), (2, 0, 1))
train_xc = x_centers[0:7, :, :].reshape(7.0*img2sh[3]*2.0)
train_yc = y_centers[0:7, :, :].reshape(7.0*img2sh[3]*2.0)
ones = np.ones([7,img2sh[3], 2] )
train_labels = ((np.asarray([0,1])).reshape(1, 1, 2)*ones).reshape(7.0*img2sh[3]*2.0)

# The validation set is the 8th realization
valid_dataset = np.transpose(imgs2[:, :, 7, :, :].reshape(img2sh[0], img2sh[1], 1.0*img2sh[3]*2.0), (2, 0, 1))
valid_xc = x_centers[7, :, :].reshape(1.0*img2sh[3]*2.0)
valid_yc = y_centers[7, :, :].reshape(1.0*img2sh[3]*2.0)
ones = np.ones([1,img2sh[3], 2] )
valid_labels = ((np.asarray([0,1])).reshape(1, 1, 2)*ones).reshape(1.0*img2sh[3]*2.0)

# The test data set is the 9th realization
test_dataset = np.transpose(imgs2[:, :, 8, :, :].reshape(img2sh[0], img2sh[1], 1.0*img2sh[3]*2.0), (2, 0, 1))
test_xc = x_centers[8, :, :].reshape(1.0*img2sh[3]*2.0)
test_yc = y_centers[8, :, :].reshape(1.0*img2sh[3]*2.0)
ones = np.ones([1,img2sh[3], 2] )
test_labels = ((np.asarray([0,1])).reshape(1, 1, 2)*ones).reshape(1.0*img2sh[3]*2.0)


/Users/jhargis/anaconda/envs/tensorflow/lib/python2.7/site-packages/ipykernel/__main__.py:37: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/jhargis/anaconda/envs/tensorflow/lib/python2.7/site-packages/ipykernel/__main__.py:47: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/jhargis/anaconda/envs/tensorflow/lib/python2.7/site-packages/ipykernel/__main__.py:51: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/jhargis/anaconda/envs/tensorflow/lib/python2.7/site-packages/ipykernel/__main__.py:60: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/jhargis/anaconda/envs/tensorflow/lib/python2.7/site-packages/ipykernel/__main__.py:64: VisibleDeprecationWarning: using a non-integer number instead of an intege

In [6]:
print "Master images tensor shape:", img2sh
print
print "Train dataset shape:", train_dataset.shape
print "Train labels shape :", train_labels.shape
print "Test dataset shape :", test_dataset.shape
print "Valid dataset shape:", valid_dataset.shape
print "TOTAL data sets    : ", train_dataset.shape[0] + test_dataset.shape[0] + valid_dataset.shape[0]

Master images tensor shape: (32, 32, 9, 19720, 2)

Train dataset shape: (276080, 32, 32)
Train labels shape : (276080,)
Test dataset shape : (39440, 32, 32)
Valid dataset shape: (39440, 32, 32)
TOTAL data sets    :  354960


In [7]:
# Reformat into a TensorFlow-friendly shape:
# - convolutions need the image data formatted as a cube (width by height by #channels)
# - labels as float 1-hot encodings.

image_size = 32
num_labels = 2
num_channels = 1 # grayscale

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

('Training set', (276080, 32, 32, 1), (276080, 2))
('Validation set', (39440, 32, 32, 1), (39440, 2))
('Test set', (39440, 32, 32, 1), (39440, 2))


In [8]:
# Cut this down to a subset
#test_frac = 0.3
#_train_dataset = train_dataset[0:train_dataset.shape[0]*test_frac,:,:,:]
#_train_labels  = train_labels[0:train_labels.shape[0]*test_frac,:]
#_test_dataset = test_dataset[0:test_dataset.shape[0]*test_frac,:,:,:]
##_test_labels  = test_labels[0:test_labels.shape[0]*test_frac,:]
#_valid_dataset = valid_dataset[0:valid_dataset.shape[0]*test_frac,:,:,:]
#_valid_labels  = valid_labels[0:valid_labels.shape[0]*test_frac,:]
#print('--> Using subset <--')
#print('Training set', _train_dataset.shape, _train_labels.shape)
#print('Validation set', _valid_dataset.shape, _valid_labels.shape)
#print('Test set', _test_dataset.shape, _test_labels.shape)
#train_dataset = _train_dataset
#train_labels = _train_labels
#test_dataset = _test_dataset
#test_labels = _test_labels
#valid_dataset = _valid_dataset
#valid_labels = _valid_labels
#print('Training set', train_dataset.shape, train_labels.shape)
#print('Validation set', valid_dataset.shape, valid_labels.shape)
#print('Test set', test_dataset.shape, test_labels.shape)

In [9]:
# Randomize the training data set
permutation = np.random.permutation(train_labels.shape[0])
train_dataset = train_dataset[permutation,:,:]
train_labels = train_labels[permutation]

In [ ]:
# Build the network
network = input_data(shape=[None, 32, 32, 1], name='input')
network = conv_2d(network, 32, 3, activation='relu', regularizer="L2", bias=True, name='conv_layer1', bias_init='truncated_normal')
network = max_pool_2d(network, 2)
network = local_response_normalization(network)
network = conv_2d(network, 64, 3, activation='relu', regularizer="L2", bias=True, name='conv_layer2', bias_init='truncated_normal')
network = max_pool_2d(network, 2)
network = local_response_normalization(network)
network = fully_connected(network, 128, activation='relu', name='fc1')
network = dropout(network, 0.8)
network = fully_connected(network, 256, activation='relu', name='fc2')
network = dropout(network, 0.8)
network = fully_connected(network, 2, activation='softmax', name='fc3')
network = regression(network, optimizer='sgd', learning_rate=0.01,
                     loss='categorical_crossentropy', name='target')

In [ ]:
# Train the network
model = tflearn.DNN(network, tensorboard_verbose=3)
model.fit({'input': train_dataset}, {'target': train_labels}, n_epoch=200000,
           validation_set=({'input': test_dataset}, {'target': test_labels}),
           snapshot_step=10000, show_metric=True, run_id='WL')

Training Step: 391  | total loss: 0.69321
| SGD | epoch: 000 | loss: 0.69321 - acc: 0.4860 -- iter: 025024/276080
